# Example 6: Fault evolution
This example describes how to visualize the evolution of faults

## Setup

1. First, you will need sign in to your Google account. If you're not signed in, you can sign in [here](https://myaccount.google.com/?utm_source=sign_in_no_continue)
2. Next, head on to the [Colab Welcome Page](https://colab.research.google.com/notebooks/welcome.ipynb#recent=true).
3. There, select Github in the top tab and search for https://github.com/thilowrona/fault_analysis_toolbox/blob/master/examples/6-evolution/6-evolution.ipynb
4. Clicking opens this notebook. This is a Jupyter notebook; an awesome combination of code and documentation allowing us work on, describe and share our projects.
5. When you run the first cell, you will face a pop-up saying ‘Warning: This notebook was not authored by Google’; you should click on ‘Run Anyway’ to get rid of the warning.
6. Next we want to save our notebook. If you click on ‘File’ and then ‘Save’, you will see a pop-up saying ´CANNOT SAVE CHANGES´. Now, click on ‘SAVE A COPY IN DRIVE’. This opens up a new tab with the same file, but this time located in your Drive. If you want to continue working after saving, use the file in the new tab. Your notebook will be saved in a folder called Colab Notebooks in your Google Drive by default.

## Load packages
To run the toolbox, we will need a couple of packages including the toolbox itself. So let's install them:

!git clone https://github.com/thilowrona/fault_analysis_toolbox

!pip3 install vtk
!pip3 install git+https://github.com/ulikoehler/cv_algorithms.git

Now we can load the python packages:

In [ ]:
import pickle

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from ipywidgets import Layout, interactive, widgets
from tqdm import tqdm


and the functions from the fault analysis toolbox that we want to use:

In [ ]:
from faultanalysistoolbox.metrics import get_fault_labels, get_fault, total_length, max_value_nodes
from faultanalysistoolbox.plots import plot_faults, get_node_colors, plot_width
from faultanalysistoolbox.utils import get_colors

## Load and plot faults

First, we load our fault networks extracted from a 2-D model over several timesteps:

In [ ]:
Gs = []
for n in range(3,50):
  # Gs.append(pickle.load(open('/content/fault_analysis_toolbox/examples/6-evolution/graphs/g_'+ str(n) + '.p', 'rb')))
  Gs.append(pickle.load(open(
        '/home/mrudolf/Nextcloud/GitRepos/fault_analysis_toolbox/examples/6-evolution/graphs/g_' + str(n) + '.p', 'rb')))


Now we can visualize these faults:

In [ ]:
%matplotlib inline
from ipywidgets import interactive, widgets, Layout


def f(time):
    fig, ax = plt.subplots(figsize=(16,4))
    plot_faults(Gs[time], ax, label=False)
    plt.xlim([1000, 3500])
    plt.ylim([600, 0])
    plt.show()

interactive_plot = interactive(f, time=widgets.IntSlider(min=3, max=49, step=1, layout=Layout(width='900px')))
output = interactive_plot.children[-1]
output.layout.width = '1000px'
interactive_plot

Note that you use the arrow keys to play the time forward and backward.

## Loop through time
One of the easiest ways to follow the evolution of our faults is to track them through time. All we need to do is:

1.   Loop through time
2.   Get fault labels
3.   Visualize them



In [ ]:
max_comp = 45

faults      = np.zeros((max_comp,len(Gs)))
faults[:,:] = np.nan

for time in range(len(Gs)):
    G = Gs[time]
    labels = get_fault_labels(G)
    number_of_faults = len(labels)
    for n, label in enumerate(labels):
      fault = get_fault(G, label)
      faults[n, time]  = label

To visualize them, we can use these nice bar plots:

In [ ]:
def bar_plot(attribute, faults, times, steps=[], ax=[]):

    colors = get_colors() 
        
    if ax==[]:
        fig, ax = plt.subplots()  

    if steps==[]:
        steps = range(attribute.shape[1])
    
    for n, step in enumerate(steps):
        bottom = 0
        for m, fault in enumerate(faults[:, step]):
            if np.isfinite(fault):
                a=attribute[m,step]
                ax.bar(n, a, 1, bottom=bottom, alpha=0.75, edgecolor='white', color=colors[int(fault),:])
                bottom += a
            else:
                break



steps = range(len(Gs))

fig, ax = plt.subplots(figsize=(16,5))
bar_plot(np.ones_like(faults), faults, len(Gs), steps, ax)
ax.set_xlim([-0.5, 45.5])
ax.set_xlabel('Time')
ax.set_ylabel('Faults')
plt.show()

Here we can see the fault labels as colors as well as the number of faults as histogram, but the plot doesn't capture the fault size.

So let's add the fault length and displacement to the mix. First, we need to track these attributes through time:

In [ ]:
max_comp = 45

faults      = np.zeros((max_comp,len(Gs)))
faults[:,:] = np.nan

lengths         = np.zeros((max_comp,len(Gs)))
lengths[:,:]    = np.nan

displacements         = np.zeros((max_comp,len(Gs)))
displacements[:,:]    = np.nan

for time in range(len(Gs)):    
    
    G = Gs[time]
    
    labels           = get_fault_labels(G)
    number_of_faults = len(labels)

    for n, label in enumerate(labels):

      fault = get_fault(G, label)

      faults[n, time]  = label    
      lengths[n, time] = total_length(fault)
      displacements[n, time] = max_value_nodes(fault, 'displacement')

    lengths[:,time] = np.sort(lengths[:,time])
    displacements[:,time] = np.sort(displacements[:,time])


Now we can plot 

In [ ]:

def stack_plot(attribute, faults, times, steps=[], ax=[]):
    
    colors = get_colors()
    
    if ax==[]:
        fig, ax = plt.subplots()  

    if steps==[]:
        steps = range(attribute.shape[1])   
    
    max_fault = int(np.nanmax(faults))

    x = np.arange(len(steps))
    y = np.zeros((max_fault, len(steps)))
        
    for N in range(max_fault):
        for n in steps:
            row = faults[:,n]
            if N in faults[:,n]:
                index = np.where(row==N)[0][0]
                y[N,n] = attribute[index,n]
            
    ax.stackplot(x, y, colors=colors[:max_fault,:], alpha=0.75, edgecolor='white', linewidth=0.5)
    

fig, ax = plt.subplots(figsize=(16,5))
stack_plot(lengths, faults, len(Gs), steps, ax)
ax.set_xlim([0, 45])
ax.set_xlabel('Time')
ax.set_ylabel('Faults weighted by length')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16,5))
stack_plot(displacements/1000, faults, len(Gs), steps, ax)
ax.set_xlim([0, 45])
ax.set_xlabel('Time')
ax.set_ylabel('Faults weighted by displacement')
plt.show()

This is great! We can summarize the evolution of the entire fault system in one plot, but what we don't see is how individual faults grow. We know they consist of multiple segments (components), which merge and split up with time. How could we keep track of these processes?

The basic idea is that we define another graph, which captures the evolution of the fault network through time. Let's start from here again:

In [ ]:
time = 0

G_0 = Gs[time]
G_1 = Gs[time+1]

fig, ax = plt.subplots(figsize=(16,4))
plot_faults(G_0, ax, label=True)
plt.title('time 0')
plt.xlim([1000, 3500])
plt.ylim([600, 0])
plt.show()

fig, ax = plt.subplots(figsize=(16,4))
plt.title('time 1')
plot_faults(G_1, ax, label=True)
plt.xlim([1000, 3500])
plt.ylim([600, 0])
plt.show()

Let's set up a graph (H), where each node corresponds to a fault segment (component) at a particular time. Moreover, we populate these nodes with some basic properties:

In [ ]:
H = nx.Graph()

def H_add_nodes(H, G, time):
  
    for cc in sorted(nx.connected_components(G)):
        for node in cc:
            label = G.nodes[node]['component']
            H.add_node((time, label))
            
            H.nodes[(time, label)]['pos'] = (time, label)
            H.nodes[(time, label)]['time'] = time
            H.nodes[(time, label)]['fault'] = G.nodes[node]['fault']
            H.nodes[(time, label)]['component'] = G.nodes[node]['component']
        
            break
    
        G_cc = nx.subgraph(G, cc)
        
        H.nodes[(time, label)]['length'] = total_length(G_cc)
        H.nodes[(time, label)]['displacement'] = max_value_nodes(G_cc, 'displacement')
        
    return H 

H = H_add_nodes(H, G_0, 0)
H = H_add_nodes(H, G_1, 1)  

Let's look at H:

In [ ]:
plt.figure(figsize=(3,6))
nx.draw(H,
        pos = nx.get_node_attributes(H, 'pos'),
        labels=nx.get_node_attributes(H, 'fault'),
        with_labels=True,
        node_color = get_node_colors(H, 'fault'))

Alright, just some nodes corresponding to the components at time 0 and 1 labelled by the fault that they belong to.

Now let's connect the components which belong to the same fault, i.e. have the same label in the above plot. For this purpose, we create a dictionary of the components belonging to each fault:

In [ ]:
def get_dictionary(G):
    faults = get_fault_labels(G)        
    dic = {}
    for fault in faults:
        G_fault = get_fault(G, fault)
        components = set()
        for node in G_fault:
            components.add(G_fault.nodes[node]['component'])
        dic[fault] = sorted(list(components))
    return dic


dic_0 = get_dictionary(G_0)
dic_1 = get_dictionary(G_1)

print(dic_0)

Now we compute the faults common between consecutive time steps:

In [ ]:
faults_0 = get_fault_labels(G_0)
faults_1 = get_fault_labels(G_1)

common_faults = list(set(faults_0).intersection(set(faults_1)))

Next we loop through the common faults, get the components for time 0 (starts) and time 1 (ends) and connect the components. This is a bit complicated because we can have different numbers of components, e.g. sometimes the fault has only one component at time 0 (len(starts)==1) and one at time 1 (len(ends)==1), then we can simply add an edge, but if there are multiple components to each fault, we need add multiple edges:

In [ ]:
for fault in common_faults:
    starts = dic_0[fault]
    ends   = dic_1[fault]
    if len(starts) == 1 and len(ends) == 1:
        H.add_edge((time, starts[0]), (time+1, ends[0]))
    elif len(starts) == 1 and len(ends) > 1:
        for end in ends:
            H.add_edge((time, starts[0]), (time+1, end))
    elif len(starts) > 1 and len(ends) == 1:
          for start in starts:
            H.add_edge((time, start), (time+1, ends[0]))
    elif len(starts) > 1 and len(ends) > 1:
        if len(starts) == len(ends):
            for start, end in zip(starts, ends):
                H.add_edge((time, start), (time+1, end))
        elif len(starts) < len(ends):
            minimum = min(len(starts), len(ends))
            difference = len(ends)-len(starts)
            for n in range(minimum):
                H.add_edge((time, starts[n]), (time+1, ends[n]))
            for n in range(difference):
                H.add_edge((time, starts[minimum-1]), (time+1, ends[minimum-1+n+1]))
        elif len(starts) > len(ends):
            minimum = min(len(starts), len(ends))
            difference = len(starts)-len(ends)                
            for n in range(minimum):
                H.add_edge((time, starts[n]), (time+1, ends[n]))                
            for n in range(difference):
                H.add_edge((time, starts[minimum-1+n+1]), (time+1, ends[minimum-1])) 

Now let's look at the result:

In [ ]:
plt.figure(figsize=(3,6))
nx.draw(H,
        pos = nx.get_node_attributes(H, 'pos'),
        labels=nx.get_node_attributes(H, 'fault'),
        with_labels=True,
        node_color = get_node_colors(H, 'fault'))

Excellent! Let's do this for all time steps:

In [ ]:
H = nx.Graph()

for time in tqdm(range(0, len(Gs)-1)):

  G_0 = Gs[time]
  G_1 = Gs[time+1]

  if time == 0:
    H = H_add_nodes(H, G_0, time)  
  H = H_add_nodes(H, G_1, time+1)  

  faults_0 = get_fault_labels(G_0)
  faults_1 = get_fault_labels(G_1) 

  dic_0 = get_dictionary(G_0)
  dic_1 = get_dictionary(G_1)
          
  
  faults = list(set(faults_0).intersection(set(faults_1)))


  for fault in faults:
      starts = dic_0[fault]
      ends   = dic_1[fault]
      if len(starts) == 1 and len(ends) == 1:
          H.add_edge((time, starts[0]), (time+1, ends[0]))
      elif len(starts) == 1 and len(ends) > 1:
          for end in ends:
              H.add_edge((time, starts[0]), (time+1, end))
      elif len(starts) > 1 and len(ends) == 1:
            for start in starts:
              H.add_edge((time, start), (time+1, ends[0]))
      elif len(starts) > 1 and len(ends) > 1:
          if len(starts) == len(ends):
              for start, end in zip(starts, ends):
                  H.add_edge((time, start), (time+1, end))
          elif len(starts) < len(ends):
              minimum = min(len(starts), len(ends))
              difference = len(ends)-len(starts)
              for n in range(minimum):
                  H.add_edge((time, starts[n]), (time+1, ends[n]))
              for n in range(difference):
                  H.add_edge((time, starts[minimum-1]), (time+1, ends[minimum-1+n+1]))
          elif len(starts) > len(ends):
              minimum = min(len(starts), len(ends))
              difference = len(starts)-len(ends)                
              for n in range(minimum):
                  H.add_edge((time, starts[n]), (time+1, ends[n]))                
              for n in range(difference):
                  H.add_edge((time, starts[minimum-1+n+1]), (time+1, ends[minimum-1]))     

In [ ]:
plt.figure(figsize=(18,8))
nx.draw(H,
        pos = nx.get_node_attributes(H, 'pos'),
        labels=nx.get_node_attributes(H, 'fault'),
        with_labels=True,
        node_color = get_node_colors(H, 'fault'))

Now we start to get a grasp of the complexity involved in the evolution of the fault system. Let's just check out one fault:

In [ ]:
fault = 10

H_sub = nx.subgraph(H, [node for node in H if H.nodes[node]['fault']==fault])

fig, ax = plt.subplots(figsize=(18,8))
nx.draw(H_sub,
        pos = nx.get_node_attributes(H_sub, 'pos'),
        labels=nx.get_node_attributes(H_sub, 'time'),
        with_labels=True,
        node_color = get_node_colors(H_sub, 'fault'),
        ax=ax)
plt.show()

Now we can see how the fault splits in two segments at time 12 and then merges again into one at time 13 and much more...

For example, we can use the H-graph to plot the evolution of fault properties, such as length and displacement through time:

In [ ]:
fault = 10
attribute_0 = 'length'
attribute_1 = 'displacement'

H_sub = nx.subgraph(H, [node for node in H if H.nodes[node]['fault']==fault])

fig, ax = plt.subplots(figsize=(12,12))
nx.draw(H_sub,
        pos = dict(zip([node for node in H_sub], [(H_sub.nodes[node][attribute_0], H_sub.nodes[node][attribute_1]) for node in H_sub])),
        labels=nx.get_node_attributes(H_sub, 'time'),
        with_labels=True,
        node_color = get_node_colors(H_sub, 'fault'),
        ax=ax)
ax.set_xlabel('Length')
ax.set_ylabel('Displacement')
ax.axis('on')
plt.show()

We can see how the fault grows sometimes in length and sometimes in displacement, how it captures smaller segements (time 12) and how it dies out (time >30) becoming shorter and shorter.

And we can plot these properties for all faults in the system:

In [ ]:
attribute_0 = 'length'
attribute_1 = 'displacement'

fig, ax = plt.subplots(figsize=(12,12))
nx.draw(H,
        pos = dict(zip([node for node in H], [(H.nodes[node][attribute_0], H.nodes[node][attribute_1]) for node in H])),
        labels=nx.get_node_attributes(H, 'fault'),
        with_labels=True,
        node_color = get_node_colors(H, 'fault'),
        ax=ax)
ax.set_xlabel('Length')
ax.set_ylabel('Displacement')
ax.axis('on')
plt.show()

Last, we want to capture these processes in Sankey-type diagram, where the edge width of our graph H shows a fault property, e.g. displacement:

In [ ]:
attribute = 'displacement'
factor= 0.00001
width = dict(zip([node for node in H], [H.nodes[node][attribute]*factor for node in H]))

fig, ax = plt.subplots(figsize=(16,8))

plot_width(H, ax, width, tips=False)

Check this out! The long-lived blue line which thickens with time is fault 10, which we can track throughout our model run:

In [ ]:
%matplotlib inline
from ipywidgets import interactive, widgets, Layout


def f(time):
    fig, ax = plt.subplots(figsize=(16,4))
    plot_faults(Gs[time], ax, label=True)
    plt.xlim([1000, 3500])
    plt.ylim([600, 0])
    plt.show()

interactive_plot = interactive(f, time=widgets.IntSlider(min=3, max=49, step=1, layout=Layout(width='900px')))
output = interactive_plot.children[-1]
output.layout.width = '1000px'
interactive_plot